# Normal versus abnormal behavior

2. Fraud detection without labels

When you can't rely on fraud labels, you can use unsupervised learning to detect suspicious behavior. Suspicious behavior is behavior that is very uncommon in your data, for example, very large transactions, or many transactions in a short period of time. Such behavior often is an indication of fraud, but of course can also just be uncommon but not fraudulent. This type of fraud detection is challenging, because you don't have trustworthy labels to check your model results against. But, in fact, not having labels is the reality for many cases of fraud detection.

3. What is normal behavior?

In order to detect suspicious behavior, you need to understand your data very well. A good exploratory data analysis, including distribution plots, checking for outliers and correlations etc, is crucial. The fraud analysts can help you understand what are normal values for your data, and also what typifies fraudulent behavior. Moreover, you need to investigate whether your data is homogeneous, or whether different types of clients display very different behavior. What is normal for one does not mean it's normal for another. For example, older age groups might have much higher total amount of health insurance claims than younger people. Or, a millionaire might make much larger transactions on average than a student. If that is the case in your data, you need to find homogeneous subgroups of data that are similar, such that you can look for abnormal behavior within subgroups.

4. Customer segmentation: normal behavior within segments

So what can you think about when checking for segments in your data? **First of all, you need to make sure all your data points are the same type**. By type I mean: are they individuals, groups of people, companies, or governmental organizations? Then, think about whether the data points differ on, for example spending patterns, age, location, or frequency of transactions. Especially for credit card fraud, **location can be a big indication for fraud**. But this also goes for e-commerce sites; where is the IP address located, and where is the product ordered to ship? If they are far apart that might not be normal for most clients, unless they indicate otherwise. Last thing to keep in mind, is that you have to create a separate model on each segment, because you want to detect suspicious behavior within each segment. But that means that you have to think about how to aggregate the many model results back into one final list.

![Alt text](image-8.png)


### Exercise

#### Exploring your data
In the next exercises, you will be looking at bank payment transaction data. The financial transactions are categorized by type of expense, as well as the amount spent. Moreover, you have some client characteristics available such as age group and gender. Some of the transactions are labelled as fraud; you'll treat these labels as given and will use those to validate the results.

When using unsupervised learning techniques for fraud detection, you want to distinguish normal from abnormal (thus potentially fraudulent) behavior. As a fraud analyst to understand what is "normal", you need to have a good understanding of the data and its characteristics. Let's explore the data in this first exercise.

In [ ]:
# Get the dataframe shape
df.shape

# Display the first 5 rows
df.head()

# Groupby categories and take the mean
print(df.groupby('category').mean())

![Alt text](image-9.png)

you can see from the category averages that fraud is more prevalent in these categories.

#### Customer segmentation
In this exercise you're going to check whether there are any obvious patterns for the clients in this data, thus whether you need to segment your data into groups, or whether the data is rather homogenous.

You unfortunately don't have a lot client information available; you can't for example distinguish between the wealth levels of different clients. However, there is data on **age ** available, so let's see whether there is any significant difference between behavior of age groups.

In [ ]:
# Group by age groups and get the mean
df.groupby('age').mean()

# Count the values of the observations in each age group
print(df['age'].value_counts())

#### Using statistics to define normal behavior
In the previous exercises we saw that fraud is more prevalent in certain transaction categories, but that there is no obvious way to segment our data into for example age groups. This time, let's investigate the average amounts spend in normal transactions versus fraud transactions. This gives you an idea of how fraudulent transactions differ structurally from normal transactions.

In [ ]:
# Create two dataframes with fraud and non-fraud data 
df_fraud = df.loc[df.fraud == 1] 
df_non_fraud = df.loc[df.fraud == 0]

# Plot histograms of the amounts in fraud and non-fraud data 
plt.hist(df_fraud.amount, alpha=0.5, label='fraud')
plt.hist(df_non_fraud.amount, alpha=0.5, label='nonfraud')
plt.legend()
plt.show()

![Alt text](image-10.png)

As the number fraud observations is much smaller, it is difficult to see the full distribution. Nonetheless, you can see that the **fraudulent transactions tend to be on the larger side relative to normal obervations**. This is good news, as it helps us later in detecting fraud from non-fraud. In the next chapter you're going to implement a clustering model to distinguish between normal and abnormal transactions, when the fraud labels are no longer available.

## Clustering methods to detect fraud

2. Clustering: trying to detect patterns in data

The objective of any clustering model is to detect patterns in your data. More specifically, to group your data in distinct clusters, that is made up of data points that are very similar to each other, but distinct from the data points in the other clusters. We can use this for fraud detection to determine which data looks very similar to the data in the clusters, and which data you would have a hard time assigning to any cluster. You can flag such data as odd, or suspicious. In this image you see a clear example where a cloud of data is clustered into three distinct clusters.
![Alt text](image-14.png)

3. K-means clustering: using the distance to cluster centroids

So, let's talk about how we achieve this pattern detecting using K-means clustering. In this example, training samples are shown as dots and cluster centroids are shown as crosses. Let's say we try to cluster the data in image A.

4. K-means clustering: using the distance to cluster centroids

We start by putting in an initial guess for two cluster centroids in figure B. You need to predefine the amount of clusters, therefore, at the start.

5. K-means clustering: using the distance to cluster centroids

You then calculate the distances of each sample in the data to the closest centroid, in figure C, which allows you to split your data into the first two clusters.

6. Step 3

And based on these initial clusters, you can refine the location of the centroids to minimize the sum of all distances in the two clusters, as you can see here in picture D.

7. Step 4

You then repeat the step of reassigning points that are nearest to the centroid, as shown in figure E, and so forth

8. Insert title here...

until it converges to the point where no sample gets reassigned to another cluster. The final clusters are depicted in picture F.
![Alt text](image-11.png)

9. K-means clustering in Python

Let's see how to implement this in Python. You begin by importing the K-means model from scikit-learn, and also a scaling method. **It is of utmost importance to scale your data before doing K-means clustering, or any algorithm that uses distances, for that matter.** If you forget to scale, features on a larger scale will weigh more heavily in the algorithm, and you don't want that. **All features should weigh equally at this point.** In the first step, you transform the data stored under df, into a NumPy array and make sure all the data is of the type float. Second, you apply the MinMaxScaler and use fit_transform on the data, as this returns the scaled data. Now you are ready to define the K-means model with 6 clusters, and fit that straight to the scaled data, as seen here. It is wise to fix the random-state, to be able to compare models.
![Alt text](image-12.png)

10. The right amount of clusters

**The drawback of K-means clustering is that you need to assign the number of clusters beforehand.** There are multiple ways to check what the right amount of clusters should be, such as the silhouette method or the elbow curve. Let's do a quick refresher on the elbow curve. **The objective of k-means is to minimize the sum of all distances between the data samples and their associated cluster centroids.** The score is the inverse of that minimization, so you want **the score to be close to zero.** By running a k-means model on clusters varying from 1 to 10, like this, and saving the scores for each model under score, you can obtain the elbow curve. Then it is a matter of simply plotting the scores against the number of clusters like this. Which results in the following plot.
![Alt text](image-15.png)

11. The elbow curve

This is an example of a typical elbow curve. The slight angle at K equals 3 suggests that 3 clusters could be optimal, although the optimal cluster number is not very pronounced in this case.
![Alt text](image-13.png)

### Exercise

#### Scaling the data
For ML algorithms using distance based metrics, it is crucial to always scale your data, as features using different scales will distort your results. K-means uses the Euclidian distance to assess distance to cluster centroids, therefore you first need to scale your data before continuing to implement the algorithm. Let's do that first.

Available is the dataframe df from the previous exercise, with some minor data preparation done so it is ready for you to use with sklearn. The fraud labels are separately stored under labels, you can use those to check the results later. numpy has been imported as np.

In [ ]:
# Import the scaler
from sklearn.preprocessing import MinMaxScaler

# Take the float values of df for X
X = df.values.astype(np.float)

# Define the scaler and apply to the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#### K-means clustering
A very commonly used clustering algorithm is K-means clustering. For fraud detection, K-means clustering is straightforward to implement and relatively powerful in predicting suspicious cases. It is a good algorithm to start with when working on fraud detection problems. However, fraud data is oftentimes very large, especially when you are working with transaction data. MiniBatch K-means is an efficient way to implement K-means on a large dataset, which you will use in this exercise.

The scaled data from the previous exercise, X_scaled is available. Let's give it a try.

In [ ]:
# Import MiniBatchKmeans 
from sklearn.cluster import MiniBatchKMeans

# Define the model 
kmeans = MiniBatchKMeans(n_clusters=8, random_state=0)

# Fit the model to the scaled data
kmeans.fit(X_scaled)

#### Elbow method
In the previous exercise you've implemented MiniBatch K-means with 8 clusters, without actually checking what the right amount of clusters should be. For our first fraud detection approach, it is important to get the number of clusters right, especially when you want to use the outliers of those clusters as fraud predictions. To decide which amount of clusters you're going to use, let's apply the Elbow method and see what the optimal number of clusters should be based on this method.

X_scaled is again available for you to use and MiniBatchKMeans has been imported from sklearn.

In [ ]:
# Define the range of clusters to try
clustno = range(1, 5)

# Run MiniBatch Kmeans over the number of clusters
kmeans = [MiniBatchKMeans(n_clusters=i, random_state=0) for i in clustno]

# Obtain the score for each model
score = [kmeans[i].fit(X_scaled).score(X_scaled) for i in range(len(kmeans))]

# Plot the models and their respective score 
plt.plot(clustno, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

![Alt text](image-16.png)

Now you can see that the optimal number of clusters should probably be at around 3 clusters, as that is where the elbow is in the curve. We'll use this in the next exercise as our baseline model, and see how well this does in detecting fraud

## Assigning fraud versus non-fraud

1. Assigning fraud versus non-fraud

So how do you go from clustering your data to fraud detection?

2. Starting with clustered data

It all starts with your optimized model, it can be k-means or any other clustering method, for that matter. In a nutshell, you're going to take the outliers of each cluster, and flag those as fraud. In this example, you're looking at three clusters.

3. Assign the cluster centroids

In a first step, you need to collect and store the cluster centroids in memory, as that is the starting point to decide what's normal and what's not.

4. Define distances from the cluster centroid

The next step is to calculate the distance of each point in the dataset, to their own cluster centroid. In this case, I use the Euclidean distance, hence you see these depicted as round circles. You then also need to define a cut-off point for the distances to define what is an outlier. You do this based on the distributions of the distances collected. Suppose you decide everything that has a bigger distance than the top 95th percentile, should be considered an outlier, ie you take the tail of the distribution of distances. In this case, that would mean that anything that falls outside the round circles, is considered an outlier.

![Alt text](image-17.png)

5. Flag fraud for those furthest away from cluster centroid

As you see in the example here, that means that you are indeed mostly flagging the odd samples that lie very far outside of the cluster centroids. These are definitely outliers and can thus be described as abnormal or suspicious. However, keep in mind that **it doesn't necessarily mean that these observations are also fraudulent.** They are, compared to the majority of normal behavior, just odd.

![Alt text](image-18.png)

6. Flagging fraud based on distance to centroid

In python, the steps to do this are exactly the steps that I've just described in pictures. It all starts with your trained clustering model, in this case, k-means. You then need to assign to which cluster each data point belongs to with the predict function, and store those results. Next, you need to save the cluster-centers with this function. Then, it's time to calculate the distance of each data point to its cluster centroid. As you can see, I use the norm function from NumPy's linear algebra package, which returns the vector norm, ie the vector of distance for each data point to their assigned cluster. Last, you use the percentiles of the distances to determine which samples are outliers. Here, I take the 93rd percentile using NumPy's percentile function, and flag it with a one if it is bigger than that. Those are the final fraud predictions.

![Alt text](image-19.png)

7. Validating your model results

Normally, this is where it gets difficult. If you don't have original fraud labels, you can't run the usual performance metrics, **hence you need some other way to sense check your results.** The best way to do so is to collaborate closely with your fraud expert, and let them have a look at the predictions and investigate further. Second, **you want to understand why these cases are outliers.** Are they truly fraudulent or just very rare cases of legit data in your sample? **If it is just a rare but non-fraudulent cases, you can avoid that by deleting certain features, or removing those cases from the data altogether.** If you do have some past cases of fraud, a good way is **to see whether your model can actually predict those when you test your model on historic data.** In the exercises, you'll use original fraud labels to check our model performance, but do keep in mind this is usually not possible.

### Exercise

#### Detecting outliers
In the next exercises, you're going to use the K-means algorithm to predict fraud, and compare those predictions to the actual labels that are saved, to sense check our results.

The fraudulent transactions are typically flagged as the observations that are furthest aways from the cluster centroid. You'll learn how to do this and how to determine the cut-off in this exercise. In the next one, you'll check the results.

Available are the scaled observations X_scaled, as well as the labels stored under the variable y.

In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)

# Define K-means model 
kmeans = MiniBatchKMeans(n_clusters=3, random_state=42).fit(X_train)

# Obtain predictions and calculate distance from cluster centroid
X_test_clusters = kmeans.predict(X_test)
X_test_clusters_centers = kmeans.cluster_centers_
dist = [np.linalg.norm(x-y) for x, y in zip(X_test, X_test_clusters_centers[X_test_clusters])]

# Create fraud predictions based on outliers on clusters 
km_y_pred = np.array(dist)
km_y_pred[dist >= np.percentile(dist, 95)] = 1
km_y_pred[dist < np.percentile(dist, 95)] = 0

#### Checking model results
In the previous exercise you've flagged all observations to be fraud, if they are in the top 5th percentile in distance from the cluster centroid. I.e. these are the very outliers of the three clusters. For this exercise you have the scaled data and labels already split into training and test set, so y_test is available. The predictions from the previous exercise, km_y_pred, are also available. Let's create some performance metrics and see how well you did.

In [ ]:
# Obtain the ROC score
print(roc_auc_score(y_test, km_y_pred))

# Create a confusion matrix
km_cm = confusion_matrix(y_test, km_y_pred)

# Plot the confusion matrix in a figure to visualize results 
plot_confusion_matrix(km_cm)

![Alt text](image-20.png)

The number of fraud cases caught increases, but false positives also increase.By lowering the threshold for cases flagged as fraud, you flag more cases overall but thereby also get more false positives.

## Other clustering fraud detection methods

Apart from K-means clustering, there are many different clustering methods, which you can use for fraud detection.

2. There are many different clustering methods

Each clustering method has its pros and cons. K-means works well when your data is clustered in normal, round shapes. As you can see in this picture, when data is clustered in very different shapes, it does not perform so well. In this picture, you see the clustering method DBSCAN performing quite well, in fact.

![Alt text](image-21.png)

3. And different ways of flagging fraud: using smallest clusters

Apart from other clustering methods, there are also other ways to flag fraud, not just based on cluster outliers. Rather than treating fraud as the oddball outlier in the existing clusters, you can also use the smallest clusters as an indication of fraud, as pictured here. You can use this approach when fraudulent behavior has commonalities, and thus will cluster together in your data. In that sense, you would expect it to cluster in tiny groups, rather than be the outliers in the larger clusters. We'll explore this more in the exercises.

4. In reality it looks more like this

The previous image was a perfect world example, but in reality, you will likely be looking at data that looks more like this. In this case, you see three obvious clusters, and a few dots that are clearly separate from the rest. As you can see, those smallest dots are outliers and outside of what you would describe as normal behavior. However, there are also medium to small clusters closely connected to the red cluster, so it's not very straightforward. In fact, **if you can visualize your data with, for example, PCA, it can be quite helpful to do so**.

![Alt text](image-22.png)

5. DBSCAN versus K-means

So let's talk a bit more about DBSCAN. **DBSCAN stands for Density-Based Spatial Clustering of Applications with Noise.** One benefit is that you do **not need to predefine the number of clusters**. The algorithm finds core samples of high density and expands clusters from them. This **works well on data which contains clusters of similar density**. This is a type of algorithm you can use to identify fraud as very small clusters. Things you do need to assign in the DBSCAN model are the maximum allowed distance between data within clusters, and the minimal number of data points in clusters. As you already saw before, **DBSCAN performs well on weirdly shaped data, but is computationally much heavier** than, for example, mini-batch K-means.

6. Implementing DBSCAN

Implementing DBSCAN is relatively straightforward. You start by defining the epsilon, eps. This is the distance between data points allowed from which the cluster expands. You also need to define the minimum samples in the cluster. Conventional DBSCAN cannot produce the optimal value of epsilon, and it requires sophisticated DBSCAN modifications to determine the optimal epsilon value automatically, which is beyond the scope of this course. You need to fit DBSCAN to your scaled data. You can use the labels function to get the assigned cluster labels for each data point. You can also count the number of clusters by counting the unique cluster labels from the cluster label predictions. I use the length of the predicted labels here to do so, but you can do this in different ways.

![Alt text](image-23.png)

7. Checking the size of the clusters

The DBSCAN model can also have performance metrics, such as **the average silhouette score**. Suppose you want to calculate the size of each cluster. You can use **NumPy's bincount function** for this. Bincount counts the number of occurrences of each value in a NumPy array, but only works on non-negative numbers. You can use this to calculate the size of each cluster. From here, you can sort on size and decide how many of the smaller clusters you want to flag as fraud. This last bit is trial and error, and will also depend on how many fraud cases the fraud team can deal with on a regular basis.

### Exercise

#### DBSCAN
In this exercise you're going to explore using a density based clustering method (DBSCAN) to detect fraud. The advantage of DBSCAN is that you do not need to define the number of clusters beforehand. Also, DBSCAN can handle weirdly shaped data (i.e. non-convex) much better than K-means can. This time, you are not going to take the outliers of the clusters and use that for fraud, but take the smallest clusters in the data and label those as fraud. You again have the scaled dataset, i.e. X_scaled available. Let's give it a try!

In [ ]:
# Import DBSCAN
from sklearn.cluster import DBSCAN

# Initialize and fit the DBSCAN model
db = DBSCAN(eps=0.9, min_samples=10, n_jobs=-1).fit(X_scaled)

# Obtain the predicted labels and calculate number of clusters
pred_labels = db.labels_
n_clusters = len(set(pred_labels)) - (1 if -1 in labels else 0)

# Print performance metrics for DBSCAN
print('Estimated number of clusters: %d' % n_clusters)
print("Homogeneity: %0.3f" % homogeneity_score(labels, pred_labels))
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_scaled, pred_labels))


> <script.py> output:

    Estimated number of clusters: 18

    Homogeneity: 0.635

    Silhouette Coefficient: 0.700

The number of clusters is much higher than with K-means. For fraud detection this is for now OK, as we are only interested in the smallest clusters, since those are considered as abnormal. Now let's have a look at those clusters and decide which one to flag as fraud.

#### Assessing smallest clusters
In this exercise you're going to have a look at the clusters that came out of DBSCAN, and flag certain clusters as fraud:

- you first need to figure out how big the clusters are, and filter out the smallest
- then, you're going to take the smallest ones and flag those as fraud
- last, you'll check with the original labels whether this does actually do a good job in detecting fraud.

Available are the DBSCAN model predictions, so n_clusters is available as well as the cluster labels, which are saved under pred_labels. Let's give it a try!

In [ ]:
# Count observations in each cluster number
counts = np.bincount(pred_labels[pred_labels>=0])

# Sort the sample counts of the clusters and take the top 3 smallest clusters
smallest_clusters = np.argsort(counts)[:3]

# Print the counts of the smallest clusters only
print("Their counts are:")      
print(counts[smallest_clusters])

#### Checking results
In this exercise you're going to check the results of your DBSCAN fraud detection model. In reality, you often don't have reliable labels and this where a fraud analyst can help you validate the results. He/She can check your results and see whether the cases you flagged are indeed suspicious. You can also check historically known cases of fraud and see whether your model flags them.

In this case, you'll use the fraud labels to check your model results. The predicted cluster numbers are available under pred_labels as well as the original fraud labels labels.

In [ ]:
# Create a dataframe of the predicted cluster numbers and fraud labels 
df = pd.DataFrame({'clusternr':pred_labels,'fraud':labels})

# Create a condition flagging fraud for the smallest clusters 
df['predicted_fraud'] = np.where((df['clusternr']==21)|(df.clusternr == 17)|(df.clusternr == 9),1 , 0)

# Run a crosstab on the results 
print(pd.crosstab(df['fraud'], df['predicted_fraud'], rownames=['Actual Fraud'], colnames=['Flagged Fraud']))

> <script.py> output:

    Flagged Fraud     0   1

    Actual Fraud           

    0              6973  16

    1               176  24
    
How does this compare to the K-means model? The good thing is: our of all flagged cases, roughly 2/3 are actually fraud! Since you only take the three smallest clusters, by definition you flag less cases of fraud, so you catch less but also have less false positives. However, you are missing quite a lot of fraud cases. Increasing the amount of smallest clusters you flag could improve that, at the cost of more false positives of course. 